# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [4]:
# Parameters
subject_id = 21096983


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:45,  1.61s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:03,  2.35s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:39,  1.66s/it]

Rendering models:  30%|███       | 9/30 [00:07<00:24,  1.17s/it]

...

Rendering models:  67%|██████▋   | 20/30 [00:07<00:04,  2.32it/s]

Rendering models:  77%|███████▋  | 23/30 [00:08<00:02,  3.17it/s]

Rendering models:  90%|█████████ | 27/30 [00:08<00:00,  4.33it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  5.59it/s]

not-logged-in-14800bc0cb17aef85ae6    0.000553
not-logged-in-f68127b03437a78136a8    0.002976
matiasd123                            0.001556
aliabdulhameed                        0.000919
not-logged-in-673e227f920b24558cc1    0.000724
sorenjensen01                         0.000639
not-logged-in-7d01fb53d3c3d3999bca    0.000515
jnarayanbvg                           0.018668
awright5                              0.002334
not-logged-in-6ead6661c7dcef165774    0.685865
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.009803
not-logged-in-a76dd994176a1ff14298    0.001405
not-logged-in-214814e52004b42491a4    0.003365
not-logged-in-9f30f411df9d28d2c541    0.001173
alexmnelson18                         0.000655
SDamian                               0.000594
megamikey0903                         0.017121
Lavadude                              0.069427
w7300953                              0.042103
w7250369                              0.001539
Theshoestargazers                     0.001814
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())